# OpenClassrooms - Ingenieur IA
# Projet 8 - Future Vision Transport 
# Participez à la conception d'une voiture autonome

## Objectif du projet : 
- **Concevoir un premier modèle de segmentation d’images**

## Plan :
- **Description et préparation des données**
    - Description et séparation des données
    - Creation des répertoires pour les données préparées ('prepared_data')
    - Remplissage des répertoires des données préparées ('prepared_data') à partir des données originales ('original_data')
    - Mapping des identifiants des catégories
    - Upload des données préparées dans Azure
        - Récupération du Workpsace Azure
        - Création du Container Azure
        - Création du Datastore Azure
        - Upload des images préparées
        - Upload des masques préparés

In [2]:
import os
import shutil
import math

from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.preprocessing.image import save_img

import numpy as np

from azureml.core import Workspace
from azure.storage.blob import BlobServiceClient
from azureml.core.datastore import Datastore
from azureml.core.authentication import ServicePrincipalAuthentication

import warnings
warnings.filterwarnings('ignore')

# Préparation des données

## Description et séparation des données
- Nous utilisons pour ce projet les données issues du dataset **Cityscape**
- Le dataset Cityscapes est un jeu de données comportant des images qui sont des photos prises d'une voiture dans différentes villes
- Le dataset contient également les annotations (aussi appelées masques) de chaque image
- Le masque est également une image dont les pixels correspondent à la catégorie dans l'image originale (piéton, nature, véhicule...)


- Le dataset contient les jeux de données suivants : **Train (2975 images et masques), Validation (500 images et masques) et Test**
- Les masques pour le jeu de Test ne sont pas disponibles, nous ne pourrons donc pas les utiliser pour évaluer le modèle final


- Pour remédier à ce problème nous allons procéder à la séparation du jeu de données de la façon suivante :
    - Nous allons créer un jeu de données **Test** à partir du jeu de données Validation original
        - Ce jeu de données **Test** nous permettra **d'évaluer le meilleur modèle final optimisé**
    - Nous allons séparer le jeu de données Train original en un jeu de données **Train** et un jeu de données **Validation** d'après un ratio TRAIN_VAL_SPLIT spécifié
        - Le jeu de données **Train** nous permettra **d'entrainer les différents modèles**
        - Le jeu de données **Validation** nous permettra **d'évaluer les différents modèles, de les comparer et d'optimiser le meilleur modèle**
    - Le jeu de données Test original n'est pas utilisé
    - Nous avons également créé des jeux de données Train échantillon et Validation échantillon afin d'entrainer les modèles intermédiaires sur un petit échantillon d'images afin de faire des tests
> **Nous avons été particulièrement attentifs à ne pas toucher au jeu de données Test afin qu'il n'y ait pas de fuite de données**    

## Creation des répertoires pour les données préparées ('prepared_data')

### Chemins des répertoires

In [3]:
#Répertoires des données originales
original_image_dir = 'I:\OCR_IA_P8\cityscape\original_data\leftImg8bit'
original_mask_dir = 'I:\OCR_IA_P8\cityscape\original_data\gtFine'

#Répertoire principal des données préparées
base_dataset_dir = 'I:\OCR_IA_P8\cityscape\prepared_data'

#Répertoires Images des données préparées
image_dataset_dir = 'I:\OCR_IA_P8\cityscape\prepared_data\images'
image_dataset_train_dir = os.path.join(image_dataset_dir, 'train')
image_dataset_train_echantillon_dir = os.path.join(image_dataset_dir, 'train_echantillon')
image_dataset_val_dir = os.path.join(image_dataset_dir, 'val')
image_dataset_val_echantillon_dir = os.path.join(image_dataset_dir, 'val_echantillon')
image_dataset_test_dir = os.path.join(image_dataset_dir, 'test')

#Répertoires Masques des données préparées
mask_dataset_dir = 'I:\OCR_IA_P8\cityscape\prepared_data\masks'
mask_dataset_train_dir = os.path.join(mask_dataset_dir, 'train')
mask_dataset_train_echantillon_dir = os.path.join(mask_dataset_dir, 'train_echantillon')
mask_dataset_val_dir = os.path.join(mask_dataset_dir, 'val')
mask_dataset_val_echantillon_dir = os.path.join(mask_dataset_dir, 'val_echantillon')
mask_dataset_test_dir = os.path.join(mask_dataset_dir, 'test')

### Création des répertoires

In [4]:
#Création du répertoire principal des données préparées
os.mkdir(base_dataset_dir)

#Création des répertoires Images des données préparées
os.mkdir(image_dataset_dir)
os.mkdir(image_dataset_train_dir)
os.mkdir(image_dataset_train_echantillon_dir)
os.mkdir(image_dataset_val_dir)
os.mkdir(image_dataset_val_echantillon_dir)
os.mkdir(image_dataset_test_dir)

#Création des répertoires Masques des données préparées
os.mkdir(mask_dataset_dir)
os.mkdir(mask_dataset_train_dir)
os.mkdir(mask_dataset_train_echantillon_dir)
os.mkdir(mask_dataset_val_dir)
os.mkdir(mask_dataset_val_echantillon_dir)
os.mkdir(mask_dataset_test_dir)

## Remplissage des répertoires des données préparées ('prepared_data') à partir des données originales ('original_data')

In [5]:
TRAIN_VAL_SPLIT_RATIO = 0.8
NB_IMG_ECH = 30

In [6]:
def prepare_data(type):
    #Type = image ou mask
    if type=="image":
        original_dir = original_image_dir
        dataset_train_dir = image_dataset_train_dir
        dataset_train_echantillon_dir = image_dataset_train_echantillon_dir
        dataset_val_dir = image_dataset_val_dir
        dataset_val_echantillon_dir = image_dataset_val_echantillon_dir
        dataset_test_dir = image_dataset_test_dir
    else:
        original_dir = original_mask_dir
        dataset_train_dir = mask_dataset_train_dir
        dataset_train_echantillon_dir = mask_dataset_train_echantillon_dir
        dataset_val_dir = mask_dataset_val_dir
        dataset_val_echantillon_dir = mask_dataset_val_echantillon_dir
        dataset_test_dir = mask_dataset_test_dir
    
    #Copie des images ou masques contenues dans les différents répertoires train (1 par ville) dans un même répertoire train 
    original_dir_train_list = os.listdir(os.path.join(original_dir, 'train'))
    i=0
    for dir in original_dir_train_list:
        for img in os.listdir(os.path.join((os.path.join(original_dir, 'train')), dir)):
            if (type=="image") or (type=="mask" and img[-12:-4]=='labelIds'):
                src = os.path.join(os.path.join((os.path.join(original_dir, 'train')), dir), img)
                dst = os.path.join(dataset_train_dir, img)#, "img_"+str(i)+".png")
                shutil.copyfile(src, dst)
                i+=1
            
    #Split des données : séparation du jeu de données train original en un jeu de données train et un jeu de données validation d'après le ratio train val spécifié        
    nb_total = len(os.listdir(dataset_train_dir))       
    nb_train = math.ceil(nb_total * TRAIN_VAL_SPLIT_RATIO)
    nb_val = nb_total - nb_train
    print(f"Nb {type} train total : {nb_total}")
    print(f"Nb {type} train : {nb_train}")
    print(f"Nb {type} validation : {nb_val}")
    i=0
    for img in os.listdir(dataset_train_dir):
        if i == nb_val:
            break
        src = os.path.join(os.path.join(dataset_train_dir, img))
        dst = os.path.join(dataset_val_dir, img)
        shutil.move(src, dst)
        i+=1

    # Création du jeu de données test à partir du jeu de données validation original
    original_dir_val_list = os.listdir(os.path.join(original_dir, 'val'))
    nb_test=0
    for dir in original_dir_val_list:
        for img in os.listdir(os.path.join((os.path.join(original_dir, 'val')), dir)):
            if (type=="image") or (type=="mask" and img[-12:-4]=='labelIds'):
                src = os.path.join(os.path.join((os.path.join(original_dir, 'val')), dir), img)
                dst = os.path.join(dataset_test_dir, img)
                shutil.copyfile(src, dst)
                nb_test+=1
    print(f"Nb {type} test : {nb_test}")

    # Renommage des images ou masques pour plus de clareté et pour respecter la demande d'utiliser des identifiants (ids)
    for i, img in enumerate(os.listdir(dataset_train_dir)):
        old_name=os.path.join(dataset_train_dir, img)
        new_name=os.path.join(dataset_train_dir, str(i)+".png")
        os.rename(old_name, new_name)
    for i, img in enumerate(os.listdir(dataset_val_dir)):
        old_name=os.path.join(dataset_val_dir, img)
        new_name=os.path.join(dataset_val_dir, str(i)+".png")
        os.rename(old_name, new_name)
    for i, img in enumerate(os.listdir(dataset_test_dir)):
        old_name=os.path.join(dataset_test_dir, img)
        new_name=os.path.join(dataset_test_dir, str(i)+".png")
        os.rename(old_name, new_name)

    # Création des jeux de données d'échantillon
    for i, img in enumerate(os.listdir(dataset_train_dir)):
        src = os.path.join(os.path.join(dataset_train_dir, img))
        dst = os.path.join(os.path.join(dataset_train_echantillon_dir, img))
        shutil.copyfile(src, dst)
        if i == NB_IMG_ECH-1:
            break
    for i, img in enumerate(os.listdir(dataset_val_dir)):
        src = os.path.join(os.path.join(dataset_val_dir, img))
        dst = os.path.join(os.path.join(dataset_val_echantillon_dir, img))
        shutil.copyfile(src, dst)
        if i == NB_IMG_ECH-1:
            break

In [7]:
prepare_data("image")

Nb image train total : 2975
Nb image train : 2380
Nb image validation : 595
Nb image test : 500


In [8]:
prepare_data("mask")

Nb mask train total : 2975
Nb mask train : 2380
Nb mask validation : 595
Nb mask test : 500


## Mapping des identifiants des catégories
- Le jeu de données Cityscape contient 8 catégories et 32 sous-catégories
- Comme cela nous est demandé nous allons travailler sur les **8 catégories** (et pas sur les 32 sous-catégories)
- Les 8 catégories sont : **void, flat, construction, object, nature, sky, human et vehicle**
- Nous allons pour cela implémenter un dictionnaire et une fonction de mapping permettant de passer des 32 sous-catégories aux 8 catégories

### Dictionnaire de mapping des catégories

In [9]:
dict_map_ids={}

#category void
dict_map_ids[0]=0
dict_map_ids[1]=0
dict_map_ids[2]=0
dict_map_ids[3]=0
dict_map_ids[4]=0
dict_map_ids[5]=0
dict_map_ids[6]=0

#category flat
dict_map_ids[7]=1
dict_map_ids[8]=1
dict_map_ids[9]=1
dict_map_ids[10]=1

#category construction
dict_map_ids[11]=2
dict_map_ids[12]=2
dict_map_ids[13]=2
dict_map_ids[14]=2
dict_map_ids[15]=2
dict_map_ids[16]=2

#category object
dict_map_ids[17]=3
dict_map_ids[18]=3
dict_map_ids[19]=3
dict_map_ids[20]=3

#category nature
dict_map_ids[21]=4
dict_map_ids[22]=4

#category sky
dict_map_ids[23]=5

#category human
dict_map_ids[24]=6
dict_map_ids[25]=6

#category vehicle
dict_map_ids[26]=7
dict_map_ids[27]=7
dict_map_ids[28]=7
dict_map_ids[29]=7
dict_map_ids[30]=7
dict_map_ids[31]=7
dict_map_ids[32]=7
dict_map_ids[33]=7
dict_map_ids[-1]=7

### Fonction de mapping

In [10]:
def img_map_ids(img_directory):
    for img in os.listdir(img_directory):
        src = os.path.join(img_directory, img)   
        #mapping des ids
        img_load = load_img(src)
        img_array_ch3 = img_to_array(img_load)
        img_array_ch1 = np.zeros([img_array_ch3.shape[0], img_array_ch3.shape[1], 1], dtype="float32")
        img_array_ch1[:,:,0] = img_array_ch3[:,:,0]
        img_array_mapped_ids = np.vectorize(dict_map_ids.__getitem__)(img_array_ch1)
        save_img(src, img_array_mapped_ids, scale=False)

### Mapping des 32 sous-catégories en 8 catégories

In [11]:
img_map_ids(mask_dataset_train_echantillon_dir)
img_map_ids(mask_dataset_val_echantillon_dir)

In [12]:
img_map_ids(mask_dataset_train_dir)
img_map_ids(mask_dataset_val_dir)

In [13]:
img_map_ids(mask_dataset_test_dir)

## Upload des données préparées dans Azure

### Récupération du Workpsace Azure : authentification utilisant le service recommandé 'Service Principal Authentication'

In [14]:
svc_pr_password = os.environ['CITYSCAPE_SPA_PASSWORD']
tenant_id = '894ad120-c276-4dfa-b218-d82b3fece6a7'
application_id = '21cf902f-1dc0-459d-b352-b7490946f6c6'

In [15]:
svc_pr = ServicePrincipalAuthentication(
        tenant_id=tenant_id,
        service_principal_id=application_id,
        service_principal_password=svc_pr_password)

In [16]:
subscription_id = subscription_id = os.environ['AZURE_SUBSCRIPTION_ID']
resource_group = 'OC-IA-P8-GPU'
workspace_name = 'WS-IA-P8-GPU'
ws = Workspace(
        subscription_id=subscription_id,
        resource_group=resource_group,
        workspace_name=workspace_name,
        auth=svc_pr)

In [17]:
print(ws.name, ws.location, ws.resource_group, sep='\t')

WS-IA-P8-GPU	eastus	OC-IA-P8-GPU


### Création du Container Azure

In [18]:
connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')

In [19]:
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_name = "cityscape-prepared"
container_client = blob_service_client.create_container(container_name)

### Création du Datastore Azure

In [20]:
blob_datastore_name='datastore_cityscape_prepared'

In [21]:
account_name=os.getenv("AZURE_ACCOUNT_NAME")
account_key=os.getenv("AZURE_ACCOUNT_KEY")

In [22]:
blob_datastore = Datastore.register_azure_blob_container(workspace=ws, 
                                                         datastore_name=blob_datastore_name, 
                                                         container_name=container_name, 
                                                         account_name=account_name,
                                                         account_key=account_key)

### Récupération du Datastore Azure

In [23]:
datastore = Datastore.get(ws, blob_datastore_name)

### Upload des images préparées

In [24]:
datastore.upload(src_dir=image_dataset_train_echantillon_dir, target_path='/prepared_data/images/train_echantillon/', overwrite=False, show_progress=True)

Uploading an estimated of 30 files
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train_echantillon\100.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train_echantillon\100.png, 1 files out of an estimated total of 30
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train_echantillon\1.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train_echantillon\1.png, 2 files out of an estimated total of 30
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train_echantillon\0.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train_echantillon\0.png, 3 files out of an estimated total of 30
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train_echantillon\1001.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train_echantillon\1001.png, 4 files out of an estimated total of 30
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train_echantillon\10.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train_echantillon\10.png, 5 files out o

$AZUREML_DATAREFERENCE_487af5a1020a4d2bb945bb3c2b7720b3

In [25]:
datastore.upload(src_dir=image_dataset_train_dir, target_path='/prepared_data/images/train/', overwrite=False, show_progress=True)

Uploading an estimated of 2380 files
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\0.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\0.png, 1 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\1.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\1.png, 2 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\10.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\10.png, 3 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\100.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\100.png, 4 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\1000.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\1000.png, 5 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\1001.png
Uploaded I:\

Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\1041.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\1041.png, 47 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\1042.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\1042.png, 48 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\1040.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\1040.png, 49 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\1043.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\1043.png, 50 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\1038.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\1038.png, 51 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\1047.png
Uploaded I:\OCR_IA_P8\city

Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\1083.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\1083.png, 93 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\1084.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\1084.png, 94 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\1076.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\1076.png, 95 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\1086.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\1086.png, 96 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\1087.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\1087.png, 97 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\1082.png
Uploaded I:\OCR_IA_P8\city

Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\1119.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\1119.png, 139 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\112.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\112.png, 140 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\1126.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\1126.png, 141 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\1124.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\1124.png, 142 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\1127.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\1127.png, 143 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\1128.png
Uploaded I:\OCR_IA_P8\c

Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\116.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\116.png, 185 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\1165.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\1165.png, 186 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\1167.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\1167.png, 187 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\1168.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\1168.png, 188 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\1162.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\1162.png, 189 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\117.png
Uploaded I:\OCR_IA_P8\ci

...

Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\838.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\838.png, 2200 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\837.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\837.png, 2201 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\836.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\836.png, 2202 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\839.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\839.png, 2203 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\84.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\84.png, 2204 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\840.png
Uploaded I:\OCR_IA_P8\citysca

Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\876.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\876.png, 2246 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\879.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\879.png, 2247 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\88.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\88.png, 2248 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\880.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\880.png, 2249 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\881.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\881.png, 2250 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\882.png
Uploaded I:\OCR_IA_P8\citysca

Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\919.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\919.png, 2292 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\920.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\920.png, 2293 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\921.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\921.png, 2294 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\924.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\924.png, 2295 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\923.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\923.png, 2296 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\922.png
Uploaded I:\OCR_IA_P8\citys

Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\960.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\960.png, 2338 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\961.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\961.png, 2339 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\962.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\962.png, 2340 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\963.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\963.png, 2341 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\964.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\train\964.png, 2342 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\train\966.png
Uploaded I:\OCR_IA_P8\citys

$AZUREML_DATAREFERENCE_6318cfc1d9fc4fb98e292fd7ca8f98f1

In [26]:
datastore.upload(src_dir=image_dataset_val_echantillon_dir, target_path='/prepared_data/images/val_echantillon/', overwrite=False, show_progress=True)

Uploading an estimated of 30 files
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val_echantillon\103.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val_echantillon\103.png, 1 files out of an estimated total of 30
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val_echantillon\102.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val_echantillon\102.png, 2 files out of an estimated total of 30
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val_echantillon\0.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val_echantillon\0.png, 3 files out of an estimated total of 30
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val_echantillon\10.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val_echantillon\10.png, 4 files out of an estimated total of 30
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val_echantillon\101.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val_echantillon\101.png, 5 files out of an estimated tot

$AZUREML_DATAREFERENCE_086a914748df4755aaf75e89d6e4b803

In [27]:
datastore.upload(src_dir=image_dataset_val_dir, target_path='/prepared_data/images/val/', overwrite=False, show_progress=True)

Uploading an estimated of 595 files
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\100.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\100.png, 1 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\104.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\104.png, 2 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\103.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\103.png, 3 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\1.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\1.png, 4 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\102.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\102.png, 5 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\101.png
Uploaded I:\OCR_IA_P8\cityscape\prepa

Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\142.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\142.png, 49 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\141.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\141.png, 50 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\143.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\143.png, 51 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\144.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\144.png, 52 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\147.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\147.png, 53 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\146.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\146.png

Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\186.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\186.png, 97 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\187.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\187.png, 98 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\182.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\182.png, 99 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\189.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\189.png, 100 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\19.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\19.png, 101 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\190.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\190.png

Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\229.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\229.png, 144 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\23.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\23.png, 145 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\222.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\222.png, 146 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\230.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\230.png, 147 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\231.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\231.png, 148 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\232.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\232.

Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\271.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\271.png, 191 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\266.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\266.png, 192 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\273.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\273.png, 193 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\274.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\274.png, 194 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\269.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\269.png, 195 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\267.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\26

...

Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\481.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\481.png, 426 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\482.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\482.png, 427 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\484.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\484.png, 428 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\486.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\486.png, 429 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\488.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\488.png, 430 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\485.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\48

Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\525.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\525.png, 473 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\524.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\524.png, 474 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\526.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\526.png, 475 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\528.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\528.png, 476 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\53.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\53.png, 477 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\527.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\527.

Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\566.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\566.png, 520 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\567.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\567.png, 521 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\568.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\568.png, 522 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\569.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\569.png, 523 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\570.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\570.png, 524 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\572.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\57

Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\77.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\77.png, 568 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\78.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\78.png, 569 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\79.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\79.png, 570 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\76.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\76.png, 571 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\75.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\75.png, 572 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\val\68.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\val\68.png, 573 f

$AZUREML_DATAREFERENCE_a3cd59df445f4a24ba0c0e02a709f499

In [28]:
datastore.upload(src_dir=image_dataset_test_dir, target_path='/prepared_data/images/test/', overwrite=False, show_progress=True)

Uploading an estimated of 500 files
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\100.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\100.png, 1 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\10.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\10.png, 2 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\104.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\104.png, 3 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\1.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\1.png, 4 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\102.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\102.png, 5 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\101.png
Uploaded I:\OCR_IA_P8\citysc

Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\140.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\140.png, 48 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\141.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\141.png, 49 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\143.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\143.png, 50 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\145.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\145.png, 51 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\144.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\144.png, 52 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\147.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\

Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\183.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\183.png, 95 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\182.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\182.png, 96 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\186.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\186.png, 97 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\185.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\185.png, 98 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\188.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\188.png, 99 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\19.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\t

Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\226.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\226.png, 142 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\227.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\227.png, 143 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\228.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\228.png, 144 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\23.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\23.png, 145 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\230.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\230.png, 146 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\231.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\imag

Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\270.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\270.png, 189 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\27.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\27.png, 190 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\269.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\269.png, 191 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\272.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\272.png, 192 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\271.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\271.png, 193 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\274.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\imag

...

Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\395.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\395.png, 330 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\396.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\396.png, 331 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\398.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\398.png, 332 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\397.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\397.png, 333 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\399.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\399.png, 334 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\4.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\imag

Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\438.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\438.png, 377 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\439.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\439.png, 378 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\44.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\44.png, 379 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\440.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\440.png, 380 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\442.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\442.png, 381 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\444.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\imag

Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\481.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\481.png, 424 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\480.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\480.png, 425 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\482.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\482.png, 426 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\483.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\483.png, 427 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\485.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\485.png, 428 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\48.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\ima

Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\73.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\73.png, 471 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\74.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\74.png, 472 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\75.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\75.png, 473 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\76.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\76.png, 474 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\78.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\78.png, 475 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\images\test\77.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\images\test\7

$AZUREML_DATAREFERENCE_e639d891607041988f13d4ce7379affc

### Upload des masques préparés

In [35]:
datastore.upload(src_dir=mask_dataset_train_echantillon_dir, target_path='/prepared_data/masks/train_echantillon/', overwrite=False, show_progress=True)

Uploading an estimated of 30 files
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train_echantillon\0.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train_echantillon\0.png, 1 files out of an estimated total of 30
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train_echantillon\1004.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train_echantillon\1004.png, 2 files out of an estimated total of 30
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train_echantillon\1002.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train_echantillon\1002.png, 3 files out of an estimated total of 30
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train_echantillon\1001.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train_echantillon\1001.png, 4 files out of an estimated total of 30
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train_echantillon\1005.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train_echantillon\1005.png, 5 files out

$AZUREML_DATAREFERENCE_1e505b50ba8a4340946bbb3cba89ccf7

In [37]:
datastore.upload(src_dir=mask_dataset_train_dir, target_path='/prepared_data/masks/train/', overwrite=False, show_progress=True)

Uploading an estimated of 2380 files
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\0.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\0.png, 1 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1.png, 2 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\10.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\10.png, 3 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\100.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\100.png, 4 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1001.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1001.png, 5 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1002.png
Uploaded I:\OCR_IA_P8\c

Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1042.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1042.png, 50 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1043.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1043.png, 51 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1041.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1041.png, 52 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1044.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1044.png, 53 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1045.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1045.png, 54 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1047.png
Uploaded I:\OCR_IA_P8\cityscape\prepa

Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1091.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1091.png, 98 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1085.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1085.png, 99 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1086.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1086.png, 100 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1088.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1088.png, 101 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1090.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1090.png, 102 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1087.png
Uploaded I:\OCR_IA_P8\cityscape\pr

Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1128.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1128.png, 146 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1129.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1129.png, 147 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\113.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\113.png, 148 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1130.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1130.png, 149 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1131.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1131.png, 150 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1133.png
Uploaded I:\OCR_IA_P8\cityscape\pr

Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1171.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1171.png, 196 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1174.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1174.png, 197 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1175.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1175.png, 198 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1176.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1176.png, 199 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\118.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\118.png, 200 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\1177.png
Uploaded I:\OCR_IA_P8\cityscape\pr

...

Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\85.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\85.png, 2215 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\850.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\850.png, 2216 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\854.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\854.png, 2217 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\852.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\852.png, 2218 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\851.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\851.png, 2219 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\855.png
Uploaded I:\OCR_IA_P8\cityscape\prepared

Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\893.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\893.png, 2263 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\90.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\90.png, 2264 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\894.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\894.png, 2265 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\895.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\895.png, 2266 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\896.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\896.png, 2267 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\9.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_d

Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\939.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\939.png, 2311 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\94.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\94.png, 2312 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\942.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\942.png, 2313 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\944.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\944.png, 2314 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\945.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\945.png, 2315 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\946.png
Uploaded I:\OCR_IA_P8\cityscape\prepared

Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\983.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\983.png, 2361 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\981.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\981.png, 2362 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\984.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\984.png, 2363 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\982.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\982.png, 2364 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\985.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\train\985.png, 2365 files out of an estimated total of 2380
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\train\986.png
Uploaded I:\OCR_IA_P8\cityscape\prepar

$AZUREML_DATAREFERENCE_a16b7ad1b8ca4bd98207892a914be045

In [36]:
datastore.upload(src_dir=mask_dataset_val_echantillon_dir, target_path='/prepared_data/masks/val_echantillon/', overwrite=False, show_progress=True)

Uploading an estimated of 30 files
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val_echantillon\100.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val_echantillon\100.png, 1 files out of an estimated total of 30
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val_echantillon\0.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val_echantillon\0.png, 2 files out of an estimated total of 30
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val_echantillon\102.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val_echantillon\102.png, 3 files out of an estimated total of 30
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val_echantillon\103.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val_echantillon\103.png, 4 files out of an estimated total of 30
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val_echantillon\104.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val_echantillon\104.png, 5 files out of an estimated total of 30

$AZUREML_DATAREFERENCE_da041c62c0e84df5b9eb77e61d88e838

In [38]:
datastore.upload(src_dir=mask_dataset_val_dir, target_path='/prepared_data/masks/val/', overwrite=False, show_progress=True)

Uploading an estimated of 595 files
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\100.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\100.png, 1 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\102.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\102.png, 2 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\103.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\103.png, 3 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\0.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\0.png, 4 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\104.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\104.png, 5 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\10.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\mas

Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\141.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\141.png, 49 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\144.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\144.png, 50 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\145.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\145.png, 51 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\146.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\146.png, 52 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\147.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\147.png, 53 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\148.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\148.png, 54 files o

Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\180.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\180.png, 98 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\185.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\185.png, 99 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\191.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\191.png, 100 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\188.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\188.png, 101 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\189.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\189.png, 102 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\195.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\195.png, 103 fil

Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\231.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\231.png, 148 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\232.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\232.png, 149 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\234.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\234.png, 150 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\235.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\235.png, 151 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\230.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\230.png, 152 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\236.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\236.png, 153 f

Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\274.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\274.png, 197 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\275.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\275.png, 198 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\277.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\277.png, 199 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\278.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\278.png, 200 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\283.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\283.png, 201 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\279.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\279.png, 202 f

...

Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\495.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\495.png, 443 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\50.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\50.png, 444 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\500.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\500.png, 445 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\505.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\505.png, 446 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\5.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\5.png, 447 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\501.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\501.png, 448 files o

Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\538.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\538.png, 492 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\543.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\543.png, 493 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\544.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\544.png, 494 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\541.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\541.png, 495 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\545.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\545.png, 496 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\547.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\547.png, 497 f

Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\582.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\582.png, 540 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\584.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\584.png, 541 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\586.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\586.png, 542 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\587.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\587.png, 543 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\59.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\59.png, 544 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\588.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\588.png, 545 fil

Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\93.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\93.png, 590 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\94.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\94.png, 591 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\95.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\95.png, 592 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\96.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\96.png, 593 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\98.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\98.png, 594 files out of an estimated total of 595
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\val\99.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\val\99.png, 595 files out of 

$AZUREML_DATAREFERENCE_df9268e380164eb49d1d673be06d2a96

In [39]:
datastore.upload(src_dir=mask_dataset_test_dir, target_path='/prepared_data/masks/test/', overwrite=False, show_progress=True)

Uploading an estimated of 500 files
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\0.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\0.png, 1 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\1.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\1.png, 2 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\10.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\10.png, 3 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\100.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\100.png, 4 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\103.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\103.png, 5 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\104.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_da

Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\140.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\140.png, 51 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\141.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\141.png, 52 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\145.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\145.png, 53 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\150.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\150.png, 54 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\146.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\146.png, 55 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\147.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\147.png

Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\187.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\187.png, 101 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\19.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\19.png, 102 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\190.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\190.png, 103 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\191.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\191.png, 104 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\192.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\192.png, 105 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\193.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\193.

Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\230.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\230.png, 149 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\232.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\232.png, 150 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\234.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\234.png, 151 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\233.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\233.png, 152 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\235.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\235.png, 153 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\239.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\23

Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\274.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\274.png, 200 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\28.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\28.png, 201 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\276.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\276.png, 202 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\286.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\286.png, 203 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\282.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\282.png, 204 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\280.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\280.

...

Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\407.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\407.png, 345 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\41.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\41.png, 346 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\406.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\406.png, 347 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\413.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\413.png, 348 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\410.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\410.png, 349 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\411.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\411.

Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\459.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\459.png, 394 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\452.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\452.png, 395 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\454.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\454.png, 396 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\453.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\453.png, 397 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\455.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\455.png, 398 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\457.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\45

Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\495.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\495.png, 442 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\497.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\497.png, 443 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\499.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\499.png, 444 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\493.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\493.png, 445 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\5.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\5.png, 446 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\50.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\50.png,

Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\91.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\91.png, 491 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\90.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\90.png, 492 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\92.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\92.png, 493 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\95.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\95.png, 494 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\93.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\93.png, 495 files out of an estimated total of 500
Uploading I:\OCR_IA_P8\cityscape\prepared_data\masks\test\94.png
Uploaded I:\OCR_IA_P8\cityscape\prepared_data\masks\test\94.png, 496 f

$AZUREML_DATAREFERENCE_7060d10d7bc042aeb815fede457a46bc